In [44]:
#Make imports
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install sklearn
!pip install pandas
!pip install numpy

import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import string
import sklearn
import numpy as np
import time
from enum import Enum

!pip install multiprocess
import multiprocess
import multiprocessing as mp
from multiprocessing import Pool, Process


import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english')) - set(['at', 'do', 'your', 'from', 'to', 'out', 'no', 'the'])

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [45]:
start = time.time()
path = '/home/manikya_varshney/Documents/Python/Yale/h01-20200818-10files/h01-20200818-153021.csv'
data = pd.read_csv(path, index_col=None, header=0, engine='python' )
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  1.15108323097229 sec


In [46]:
keywords_Imp = ['stay at home' , 'do your part', 'Responsible', 
            'home', 'house', 'cancel', 'shutdown', 'postpone',
            'school closure', 'Closure', 'business closure',
            'suspension', 'quarantine', 'lockdown', 'social distance', 
            'social distancing', 'self quarantine', 'isolat', '6-feet',
            'distance', '#clubquarantine', '#quarantinelife', '#quarantineacitivites']

keywords_Ada = ['school from home' , 'learn', 'remote', 'school food service', 
            'online shopping', 'online purchase', 'online church', 'delivery',
            'drive thru', 'to go', 'take out', 'Tiktok', 'Netflix', 'telework', 
            'zoom', 'telehealth', 'telemedicine', 'work from home', 'wfh',
            'working at home', 'working remotely', 'online meeting']

keywords_Ne = ['bored' , 'lonely', 'stress', 
            'anxiety', 'scared', 'worry', 'end', 'cabin fever',
            '#sideeffectsofquarantinelife', 'tissue paper', 'toilet paper']

keywords_Sd = ['social functions' , 'gathering', 'empty streets', 
            'interaction', 'large', 'no cars', 'non-essential',
            'travel', 'unnecessary', 'crowd']

keywords_Purp = ['Flatten the curve' , 'Slow the spread', 'slow transmission', 
            'protect', 'save', '#stayhomesavelives']

keywords_Pe = ['silver lining' , 'optimistic', 'hope', 
            'bright side', 'Safe', '#togetherapart']

In [47]:
# Constants
#FINAL_COL_NAME = "FINAL_TEXT"
num_cores = mp.cpu_count()
FINAL_COL_NAME = "QT_text"

In [48]:
def keywords_cleaning(keywords_list):
    
    #Convert to lower
    for i in range(len(keywords_list)): 
        keywords_list[i] = keywords_list[i].lower()
    
    #Remove punctuations
    for i in range(len(keywords_list)):
        keywords_list[i] = keywords_list[i].translate(str.maketrans('','',string.punctuation))
    
    #More cleaning
    for i in range(len(keywords_list)):
        keywords_list[i] = keywords_list[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

    #Remove stop words
    def remove_stopwords(data):
        output_array=[]
        for sentence in data:
            temp_list=[]
            for word in sentence.split():
                if word not in stop_words:
                    temp_list.append(word)
            output_array.append(' '.join(temp_list))
        return output_array

    keywords_list=remove_stopwords(keywords_list)

    #Stemming
    ps = PorterStemmer()
    keywords_list_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_list]
    keywords_list_final = [" ".join(sentence) for sentence in keywords_list_stem]

    return keywords_list_final


In [49]:
# Enum for facets
class Facets(Enum):
    IMPLEMENTATION = "Imp"
    ADAPTATION = "Ada"
    NEGATIVE_EMOTIONS = "Ne"
    SOCIAL_DISRUPTION = "Sd"
    PURPOSE = "Purp"
    POSITIVE_EMOTION = "Pe"

In [94]:
def fuzzy_logic(args):
    row, FINAL_COL_NAME, keywords, facet, qwargs = args
    print(**qwargs['choices'])
    keyword_match, score = row[FINAL_COL_NAME].apply(process.extractOne, **qwargs)
    #keyword_match, score = process.extractOne(row[FINAL_COL_NAME], keywords, scorer = fuzz.partial_ratio)
    row['final_score_{}'.format(facet.value)] = score
    row['final_keyword_match_{}'.format(facet.value)] = keyword_match
    return row

In [95]:
def keep_only_highest(data, high_value, facet):
    data['final_score_{}'.format(facet.value)] = data['final_score_{}'.format(facet.value)].astype(int)    
    data = data[data['final_score_{}'.format(facet.value)] == 100].reset_index(drop=True)    
    return data

In [96]:
def proportion(interim_data):
    denominator = interim_data.shape[0]
    numerator = interim_data.shape[0]
    prop_val = (numerator/denominator)
    return prop_val

In [97]:
def split_dataframe(df, nums = 4): 
    chunks = list()
    num_chunks = nums
    chunk_size = len(df) // nums
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

In [98]:
def multiprocessing_(data_, keywords_, facet_, **qwargs):
    print(data_.shape)
    start = time.time()
    pool = Pool(num_cores)
    df = split_dataframe(data_, num_cores)
    tuples = [(x, FINAL_COL_NAME, keywords_, facet_, qwargs) for x in df]
    print(len(tuples))
    #data1 = pd.concat(pool.map(fuzzy_logic, tuples))
    #data1 = pool.starmap(test, [(1,2,3)])
    data1 = pool.map(fuzzy_logic, tuples)
    pool.close()
    pool.join()
    end = time.time()
    print("time = ", end - start)
    return data1
multiprocessing_(data, keywords_Imp, Facets.IMPLEMENTATION, choices = keywords_Imp, scorer = fuzz.partial_ratio)

(4737, 79)
8


TypeError: print() argument after ** must be a mapping, not list

### Keywords Cleaning

In [54]:
keywords_Imp = keywords_cleaning(keywords_Imp)
keywords_Ada = keywords_cleaning(keywords_Ada)
keywords_Ne = keywords_cleaning(keywords_Ne)
keywords_Sd = keywords_cleaning(keywords_Sd)
keywords_Purp = keywords_cleaning(keywords_Purp)
keywords_Pe = keywords_cleaning(keywords_Pe)

### Drop Rows with Empty Tweets

In [55]:
nan_value = float("NaN")
data[FINAL_COL_NAME].replace("", nan_value, inplace=True)
data.dropna(subset = [FINAL_COL_NAME], inplace=True)

# 1_Implementation

In [56]:
interim_Imp = multiprocessing_(data, keywords_Imp, Facets.IMPLEMENTATION)
#interim_Imp = data.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Imp, Facets.IMPLEMENTATION))
#interim_Imp = keep_only_highest(interim_Imp, 100, Facets.IMPLEMENTATION)

# 2_Adaptation

In [ ]:
interim_Ada = data.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Ada, Facets.ADAPTATION))
interim_Ada = keep_only_highest(interim_Ada, 100, Facets.ADAPTATION)

# 3_Negative Emotion

In [ ]:
interim_Ne = data.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Ne, Facets.NEGATIVE_EMOTIONS))
interim_Ne = keep_only_highest(interim_Ne, 100, Facets.NEGATIVE_EMOTIONS)

# 4_Social Disruption

In [ ]:
interim_Sd = data.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Sd, Facets.SOCIAL_DISRUPTION))
interim_Sd = keep_only_highest(interim_Sd, 100, Facets.SOCIAL_DISRUPTION)

# 5_Purpose

In [ ]:
interim_Purp = data.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Purp, Facets.PURPOSE))
interim_Purp = keep_only_highest(interim_Purp, 100, Facets.PURPOSE)

# 6_Positive Emotion

In [ ]:
interim_Pe = data.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Pe, Facets.POSITIVE_EMOTION))
interim_Pe = keep_only_highest(interim_Pe, 100, Facets.POSITIVE_EMOTION)

### All Proportions

In [ ]:
print("For IMPLEMENTATION:", proportion(interim_Imp))
print("For ADAPTATION:", proportion(interim_Ada))
print("For NEGATIVE EMOTIONS:", proportion(interim_Ne))
print("For SOCIAL DISRUPTION:", proportion(interim_Sd))
print("For PURPOSE:", proportion(interim_Purp))
print("For POSITIVE EMOTION:", proportion(Pe))